In [2]:
# RNNの訓練方法
# RNNの訓練は一般的なNNと同様に誤差逆伝播によって行われる
# 時系列に沿ってアンロールしたこの方法をBPTT(通時鉄器逆伝播)という

#以下お決まり設定
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [3]:
# MNISTを例にRNNの訓練を実施してみる(簡単な例としてMNISTを採用しただだけでMNISTならCNNの方が最適)

import tensorflow as tf
reset_graph()

n_steps = 28 # ステップ数(シーケンス数)
n_inputs = 28 # 特徴量数
n_neurons = 150 # 再帰ニューロン数
n_outputs = 10 # 最終的なクラス分類数

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None]) # ターゲットクラスのプレースホルダーを定義

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                          logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.


In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [5]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [6]:
X_test = X_test.reshape((-1, n_steps, n_inputs))

In [7]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Last batch accuracy:", acc_batch, "Test accuracy:", acc_test)

0 Last batch accuracy: 0.9533333 Test accuracy: 0.9288
1 Last batch accuracy: 0.96 Test accuracy: 0.9471
2 Last batch accuracy: 0.96 Test accuracy: 0.9499
3 Last batch accuracy: 0.96 Test accuracy: 0.9563
4 Last batch accuracy: 0.98 Test accuracy: 0.9677
5 Last batch accuracy: 0.93333334 Test accuracy: 0.9651
6 Last batch accuracy: 0.98 Test accuracy: 0.9685
7 Last batch accuracy: 0.96666664 Test accuracy: 0.9678
8 Last batch accuracy: 0.97333336 Test accuracy: 0.9693
9 Last batch accuracy: 0.99333334 Test accuracy: 0.9714
10 Last batch accuracy: 0.98 Test accuracy: 0.9752
11 Last batch accuracy: 0.9866667 Test accuracy: 0.9743
12 Last batch accuracy: 0.94666666 Test accuracy: 0.9716
13 Last batch accuracy: 0.97333336 Test accuracy: 0.9658
14 Last batch accuracy: 1.0 Test accuracy: 0.9772
15 Last batch accuracy: 0.98 Test accuracy: 0.974
16 Last batch accuracy: 0.99333334 Test accuracy: 0.9779
17 Last batch accuracy: 0.9866667 Test accuracy: 0.9775
18 Last batch accuracy: 0.9866667 Tes